<a href="https://colab.research.google.com/github/alihaydarkir/fine-tuning/blob/main/duznelenmis_finetunning_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# QLoRA KURULUMU - Memory problemi çözümü
!pip install peft bitsandbytes accelerate

print("🔧 QLoRA kütüphaneleri kuruluyor...")

🔧 QLoRA kütüphaneleri kuruluyor...


# BÖLÜM 1: Kütüphane İmportları

In [3]:
# BÖLÜM 1: KÜTÜPHANE İMPORTLARI
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import os
from tqdm import tqdm
import gc

# Google Colab için
from google.colab import drive

# GPU setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Cihaz: {device}")
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name()}")

print("✅ Kütüphaneler yüklendi!")

🖥️ Cihaz: cpu
✅ Kütüphaneler yüklendi!


# BÖLÜM 2: Konfigürasyon

In [4]:
# BÖLÜM 2: KONFİGÜRASYON
class Config:
    # Model ayarları
    model_name = "sberbank-ai/mGPT"    # Türkçe destekli
    max_length = 256                   # Memory için küçük

    # Eğitim ayarları
    batch_size = 1                     # Memory için minimum
    learning_rate = 2e-5               # Stable learning rate
    num_epochs = 1                     # Hızlı test için

    # Dosya yolları
    json_file = "/content/drive/MyDrive/Colab Notebooks/fine-tuning/sigorta_qa_ft.json"
    save_path = "/content/drive/MyDrive/Colab Notebooks/fine-tuning/mgpt_insurance_model"

    # Generation ayarları
    temperature = 0.7
    max_new_tokens = 50

config = Config()
print("✅ Konfigürasyon hazır!")

✅ Konfigürasyon hazır!


# BÖLÜM 3: Dataset Class

In [5]:
# BÖLÜM 3: DATASET CLASS
class InsuranceDataset(Dataset):
    def __init__(self, json_file_path, tokenizer, max_length=256):
        with open(json_file_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.max_length = max_length
        print(f"📊 {len(self.data)} örnek yüklendi")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = f"Soru: {item['input']}\nCevap: {item['output']}"

        encoding = self.tokenizer(
            text, truncation=True, padding='max_length',
            max_length=self.max_length, return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

print("✅ Dataset class hazır!")

✅ Dataset class hazır!


# BÖLÜM 4: Model ve Tokenizer

In [6]:
# BİTSANDBYTES KURULUMU DÜZELTMESİ
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U peft

print("✅ Kütüphaneler güncellendi!")
print("⚠️ Eğer hala hata alırsan runtime restart gerekebilir!")

✅ Kütüphaneler güncellendi!
⚠️ Eğer hala hata alırsan runtime restart gerekebilir!


In [7]:
# BÖLÜM 4 DÜZELTİLMİŞ VERSİYON - Doğru target modules
from peft import LoraConfig, get_peft_model, TaskType
from transformers import BitsAndBytesConfig

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print("🔤 Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("🤖 Model yükleniyor (4-bit)...")
model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# mGPT için doğru target modules
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],  # mGPT için doğru isimler
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

print("🎯 LoRA uygulanıyor...")
model = get_peft_model(model, lora_config)
print("✅ QLoRA başarıyla uygulandı!")

🔤 Tokenizer yükleniyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


🤖 Model yükleniyor (4-bit)...


🎯 LoRA uygulanıyor...
✅ QLoRA başarıyla uygulandı!


# küçük modül dene

# BÖLÜM 5: Generation Function

In [8]:
# mGPT'YE GERİ DÖN
config.model_name = "sberbank-ai/mGPT"  # Türkçe destekli

print("🔄 mGPT'ye geri dönülüyor...")
print(f"Model: {config.model_name}")

# Memory temizliği
import gc
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

print("🔤 mGPT Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("🤖 mGPT Model yükleniyor...")
model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

print("✅ mGPT hazır!")

# Hızlı Türkçe test
print("\n🧪 Türkçe Test:")
def simple_generate(question):
    prompt = f"Soru: {question}\nCevap:"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=len(inputs[0]) + 30, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "").strip()

answer = simple_generate("Sigorta nedir?")
print(f"🤖 {answer}")

🔄 mGPT'ye geri dönülüyor...
Model: sberbank-ai/mGPT
🔤 mGPT Tokenizer yükleniyor...
🤖 mGPT Model yükleniyor...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ mGPT hazır!

🧪 Türkçe Test:
🤖 Sigorta, sigortalıya, sigortalıya ait olan ve sigortalıya ait olmayan malların, sigortalıya ait


# BÖLÜM 6: Training Function

In [9]:
# SÜPER OPTİMİZE EĞİTİM - CPU için
print("⚡ CPU-optimize eğitim başlıyor...")

# Çok küçük config
config.batch_size = 1
config.max_length = 128  # 256'dan 128'e düşür
config.num_epochs = 1

# Dataset oluştur (kısa)
dataset = InsuranceDataset(config.json_file, tokenizer, config.max_length)
print(f"📊 {len(dataset)} örnek (kısa format)")

# Sadece ilk 10 örnek ile test et
from torch.utils.data import Subset
small_dataset = Subset(dataset, range(10))  # Sadece 10 örnek
dataloader = DataLoader(small_dataset, batch_size=1)

print(f"🧪 Test eğitimi: {len(small_dataset)} örnek")

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
model.train()

# Eğitim
for epoch in range(1):
    for i, batch in enumerate(dataloader):
        print(f"Batch {i+1}/10 işleniyor...")

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"✅ Batch {i+1} - Loss: {loss.item():.4f}")

        if i >= 2:  # Sadece 3 batch test et
            break

print("🎉 Test eğitimi tamamlandı!")

⚡ CPU-optimize eğitim başlıyor...
📊 89 örnek yüklendi
📊 89 örnek (kısa format)
🧪 Test eğitimi: 10 örnek
Batch 1/10 işleniyor...


KeyboardInterrupt: 